In [1]:
%cd ..

/home/kimsin98/Adolescent/StructFunc


In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import numpy as np
import pandas as pd
idx = pd.IndexSlice
SEED = 69

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style='ticks')
colors = sns.color_palette()

In [3]:
import joblib
from scipy import stats
from utils import abcd

import multiverse_sCCA

# Multiverse Summary

In [4]:
summary = pd.read_csv(multiverse_sCCA.out_path / 'summary.csv', index_col=0)
test = pd.read_csv(multiverse_sCCA.out_path / 'test.csv', index_col=0)

In [5]:
summary.join(test).sort_values('SC_test', ascending=False)

,SC,FC,FCSC,SC_test,p_SC
full-r-dev-famd-noout,0.213125,0.644793,0.621385,0.190832,1.00
full-r-dev-fa-all,0.131392,0.717416,0.717416,0.168315,0.99
full--dev-fa-all,0.131179,0.695816,0.695816,0.167222,1.00
full--dev-famd-noout,0.192345,0.644411,0.626300,0.152638,1.00
full-m-dev-famd-noout,0.144313,0.629797,0.623402,0.152148,1.00
...,...,...,...,...,...
full-m-devsoft-famd-noout,0.133647,0.641915,0.642229,0.036502,1.00
base--dev-famd-all,0.091474,0.695816,0.695816,0.035009,1.00
base-m-dev-famd-all,0.091476,0.695816,0.695816,0.034938,1.00
full--devsoft-famd-all,0.078980,0.679098,0.679098,0.027602,1.00


In [6]:
series = test['SC_test'].copy()
index = pd.MultiIndex.from_tuples(series.index.str.split('-', expand=True),
                                  names=['shell', 'covar', 'scan', 'scon', 'out'])
series.index = index

In [7]:
covar_order = {'':0, 'r':1, 'm':2, 'mr':3}
def covar_key(index):
    return pd.Index([covar_order[i] for i in index],
                    name=index.name)

SC_test_2d = (series.unstack(['scan', 'covar'])
              .sort_index(axis=1, level='covar', key=covar_key)
              .sort_index(axis=1, level='scan', sort_remaining=False)
              .rename(columns={'':'-', 'm':'M', 'mr':'M, R', 'r':'R', 'devsoft':'dev, soft'},
                      index={'fa': 'FA', 'famd':'FA, MD'}))

In [8]:
styles = []
styles.append({
    'selector': '.index_name',
    'props': 'font-style: italic; color: black; background-color: darkgrey;'
})
styles.append({
    'selector': 'th:not(.index_name)',
    'props': 'text-align: center; color: black; background-color: white;'
})
styles.append({
    'selector': 'th.row_heading',
    'props': 'border-top: 1px solid black;'
})
styles.append({
    'selector': 'th.col_heading',
    'props': 'border-left: 1px solid black;'
})
for c in range(len(SC_test_2d.columns)):
    styles.append({
        'selector': f'th.blank.col{c}',
        'props': 'border-left: 1px solid black;'
    })

(SC_test_2d.style.format(precision=3).set_table_styles(styles)
 .set_properties(**{'width':'4em', 'text-align':'center'})
 .background_gradient(cmap='OrRd', vmin=0, vmax=0.5))

# Weights

* full shell
* harmonize scanner device only
* SC motion not confound

In [9]:
fcon, _ = abcd.load_fcon()
sconfull, _ = abcd.load_scon(full=True, metrics=['fa', 'md'])

scon_tracts = abcd.get_scon_descriptions()

## FA, MD

In [10]:
scca_SC = joblib.load(multiverse_sCCA.out_path / 'full-r-dev-famd-noout' / 'scca_SC.joblib.gz')

SC_weights = pd.Series(scca_SC.x_weights_[:, 0], index=sconfull.columns)
dFC_weights = pd.Series(scca_SC.y_weights_[:, 0], index=fcon.columns)

In [11]:
SC_weights.loc[~np.isclose(SC_weights, 0)].sort_values(ascending=False, key=np.abs)

dmri_dtifullfa_fiberat_atrlh       -0.403356
dmri_dtifullfa_fiberat_unclh       -0.400079
dmri_dtifullmd_fiberat_fxlh        -0.361196
dmri_dtifullfa_fiberat_fxcutrh     -0.339231
dmri_dtifullmd_fiberat_fxrh        -0.312921
dmri_dtifullfa_fiberat_allfcclh    -0.264166
dmri_dtifullfa_fiberat_atrrh       -0.233021
dmri_dtifullfa_fiberat_fxrh        -0.230692
dmri_dtifullfa_fiberat_allfiblh    -0.219494
dmri_dtifullmd_fiberat_fxcutlh     -0.160152
dmri_dtifullfa_fiberat_ifolh       -0.150858
dmri_dtifullmd_fiberat_fxcutrh     -0.148193
dmri_dtifullfa_fiberat_allfibers   -0.137615
dmri_dtifullmd_fiberat_fmaj        -0.087420
dmri_dtifullfa_fiberat_ilflh       -0.073031
dmri_dtifullfa_fiberat_allfccrh    -0.056073
dmri_dtifullfa_fiberat_allfibrh    -0.047262
dmri_dtifullfa_fiberat_cgclh       -0.019538
dmri_dtifullmd_fiberat_cc          -0.014357
dmri_dtifullfa_fiberat_fmin        -0.007406
dtype: float64

In [12]:
dFC_weights.loc[~np.isclose(dFC_weights, 0)].sort_values(ascending=False, key=np.abs)

rsfmri_c_ngd_dt_ngd_dt            0.505117
rsfmri_c_ngd_fo_ngd_fo            0.383744
rsfmri_c_ngd_dt_ngd_cgc          -0.328813
rsfmri_c_ngd_rspltp_ngd_cgc      -0.289789
rsfmri_c_ngd_smh_ngd_ad          -0.287009
rsfmri_c_ngd_dla_ngd_dt          -0.284684
rsfmri_c_ngd_ad_ngd_ad           -0.254717
rsfmri_c_ngd_sa_ngd_rspltp       -0.241679
rsfmri_c_ngd_fo_ngd_cgc          -0.183897
rsfmri_c_ngd_rspltp_ngd_dla      -0.132839
rsfmri_c_ngd_smh_ngd_smh         -0.125316
rsfmri_c_ngd_vta_ngd_smh         -0.121355
rsfmri_c_ngd_smm_ngd_ad          -0.111522
rsfmri_c_ngd_vta_ngd_dt           0.109670
rsfmri_c_ngd_vta_ngd_ad          -0.093858
rsfmri_c_ngd_rspltp_ngd_rspltp    0.055579
rsfmri_c_ngd_ca_ngd_ca            0.023115
dtype: float64

In [13]:
scca_SC = joblib.load(multiverse_sCCA.out_path / 'full--dev-famd-noout' / 'scca_SC.joblib.gz')

SC_weights = pd.Series(scca_SC.x_weights_[:, 0], index=sconfull.columns)
dFC_weights = pd.Series(scca_SC.y_weights_[:, 0], index=fcon.columns)

In [14]:
SC_weights.loc[~np.isclose(SC_weights, 0)].sort_values(ascending=False, key=np.abs)

dmri_dtifullfa_fiberat_atrlh       -0.428699
dmri_dtifullfa_fiberat_unclh       -0.394064
dmri_dtifullfa_fiberat_fxcutrh     -0.378167
dmri_dtifullfa_fiberat_fxrh        -0.298440
dmri_dtifullfa_fiberat_atrrh       -0.283180
dmri_dtifullfa_fiberat_allfcclh    -0.263435
dmri_dtifullfa_fiberat_ifolh       -0.252019
dmri_dtifullfa_fiberat_allfiblh    -0.242809
dmri_dtifullmd_fiberat_fxlh        -0.193613
dmri_dtifullfa_fiberat_allfibers   -0.188747
dmri_dtifullmd_fiberat_fxrh        -0.159831
dmri_dtifullfa_fiberat_allfccrh    -0.125233
dmri_dtifullfa_fiberat_allfibrh    -0.117200
dmri_dtifullfa_fiberat_ilflh       -0.111078
dmri_dtifullmd_fiberat_fmaj        -0.085436
dmri_dtifullfa_fiberat_fmin        -0.081813
dmri_dtifullfa_fiberat_cgclh       -0.047592
dmri_dtifullmd_fiberat_fxcutlh     -0.009987
dmri_dtifullfa_fiberat_uncrh       -0.004717
dtype: float64

In [15]:
dFC_weights.loc[~np.isclose(dFC_weights, 0)].sort_values(ascending=False, key=np.abs)

rsfmri_c_ngd_fo_ngd_fo         0.545576
rsfmri_c_ngd_dt_ngd_dt         0.430201
rsfmri_c_ngd_rspltp_ngd_cgc   -0.329214
rsfmri_c_ngd_smh_ngd_ad       -0.276378
rsfmri_c_ngd_sa_ngd_rspltp    -0.247182
rsfmri_c_ngd_rspltp_ngd_dla   -0.212433
rsfmri_c_ngd_ad_ngd_ad        -0.201310
rsfmri_c_ngd_dt_ngd_cgc       -0.196034
rsfmri_c_ngd_dla_ngd_dt       -0.190625
rsfmri_c_ngd_fo_ngd_cgc       -0.176423
rsfmri_c_ngd_vta_ngd_smh      -0.135574
rsfmri_c_ngd_vta_ngd_ad       -0.126891
rsfmri_c_ngd_vta_ngd_dt        0.119493
rsfmri_c_ngd_vs_ngd_fo        -0.111540
rsfmri_c_ngd_smm_ngd_ad       -0.101783
rsfmri_c_ngd_smh_ngd_smh      -0.078296
rsfmri_c_ngd_vs_ngd_ca        -0.046249
rsfmri_c_ngd_smm_ngd_ca        0.005146
rsfmri_c_ngd_ca_ngd_ca         0.002356
dtype: float64

In [16]:
scca_SC = joblib.load(multiverse_sCCA.out_path / 'full-r-dev-famd-all' / 'scca_SC.joblib.gz')

SC_weights = pd.Series(scca_SC.x_weights_[:, 0], index=sconfull.columns)
dFC_weights = pd.Series(scca_SC.y_weights_[:, 0], index=fcon.columns)

In [17]:
SC_weights.loc[~np.isclose(SC_weights, 0)].sort_values(ascending=False, key=np.abs)

dmri_dtifullmd_fiberat_cstlh    0.749688
dmri_dtifullmd_fiberat_cc       0.466658
dmri_dtifullmd_fiberat_cstrh    0.430917
dmri_dtifullmd_fiberat_fxlh     0.185768
dtype: float64

In [18]:
dFC_weights.loc[~np.isclose(dFC_weights, 0)].sort_values(ascending=False, key=np.abs)

rsfmri_c_ngd_dt_ngd_cgc    0.663990
rsfmri_c_ngd_dt_ngd_dt    -0.613902
rsfmri_c_ngd_dla_ngd_dt    0.421635
rsfmri_c_ngd_fo_ngd_cgc    0.066825
dtype: float64

In [19]:
scca_SC = joblib.load(multiverse_sCCA.out_path / 'full--dev-famd-all' / 'scca_SC.joblib.gz')

SC_weights = pd.Series(scca_SC.x_weights_[:, 0], index=sconfull.columns)
dFC_weights = pd.Series(scca_SC.y_weights_[:, 0], index=fcon.columns)

In [20]:
SC_weights.loc[~np.isclose(SC_weights, 0)].sort_values(ascending=False, key=np.abs)

dmri_dtifullmd_fiberat_cstlh    0.706869
dmri_dtifullmd_fiberat_cc       0.507192
dmri_dtifullmd_fiberat_cstrh    0.470000
dmri_dtifullmd_fiberat_fxlh     0.148969
dtype: float64

In [21]:
dFC_weights.loc[~np.isclose(dFC_weights, 0)].sort_values(ascending=False, key=np.abs)

rsfmri_c_ngd_dt_ngd_cgc     0.681127
rsfmri_c_ngd_dt_ngd_dt     -0.601575
rsfmri_c_ngd_dla_ngd_dt     0.413724
rsfmri_c_ngd_fo_ngd_cgc     0.051709
rsfmri_c_ngd_vta_ngd_cgc    0.018216
dtype: float64

# FA

In [22]:
scca_SC = joblib.load(multiverse_sCCA.out_path / 'full-r-dev-fa-all' / 'scca_SC.joblib.gz')

SC_weights = pd.Series(scca_SC.x_weights_[:, 0], index=sconfull.columns[:len(scon_tracts)])
dFC_weights = pd.Series(scca_SC.y_weights_[:, 0], index=fcon.columns)

In [23]:
SC_weights.loc[~np.isclose(SC_weights, 0)].sort_values(ascending=False, key=np.abs)

dmri_dtifullfa_fiberat_allfcclh   -0.930917
dmri_dtifullfa_fiberat_allfiblh   -0.365231
dtype: float64

In [24]:
dFC_weights.loc[~np.isclose(dFC_weights, 0)].sort_values(ascending=False, key=np.abs)

rsfmri_c_ngd_smh_ngd_ad       -0.469351
rsfmri_c_ngd_smm_ngd_smh      -0.444388
rsfmri_c_ngd_rspltp_ngd_cgc   -0.432949
rsfmri_c_ngd_ca_ngd_ad         0.291039
rsfmri_c_ngd_fo_ngd_fo         0.262023
rsfmri_c_ngd_smm_ngd_ca        0.226784
rsfmri_c_ngd_smh_ngd_smh      -0.194107
rsfmri_c_ngd_fo_ngd_dla        0.174111
rsfmri_c_ngd_sa_ngd_rspltp    -0.171565
rsfmri_c_ngd_ad_ngd_ad        -0.158278
rsfmri_c_ngd_smh_ngd_dla       0.158089
rsfmri_c_ngd_smm_ngd_ad       -0.129273
rsfmri_c_ngd_vs_ngd_smh        0.091324
rsfmri_c_ngd_sa_ngd_fo        -0.064278
rsfmri_c_ngd_smh_ngd_rspltp   -0.062190
rsfmri_c_ngd_vs_ngd_smm        0.057772
rsfmri_c_ngd_rspltp_ngd_ca    -0.055165
rsfmri_c_ngd_vta_ngd_vta       0.039634
rsfmri_c_ngd_vta_ngd_cgc       0.036401
rsfmri_c_ngd_smh_ngd_ca        0.013984
dtype: float64

In [25]:
scca_SC = joblib.load(multiverse_sCCA.out_path / 'full--dev-fa-all' / 'scca_SC.joblib.gz')

SC_weights = pd.Series(scca_SC.x_weights_[:, 0], index=sconfull.columns[:len(scon_tracts)])
dFC_weights = pd.Series(scca_SC.y_weights_[:, 0], index=fcon.columns)

In [26]:
SC_weights.loc[~np.isclose(SC_weights, 0)].sort_values(ascending=False, key=np.abs)

dmri_dtifullfa_fiberat_allfcclh   -0.947912
dmri_dtifullfa_fiberat_allfiblh   -0.316998
dmri_dtifullfa_fiberat_atrlh      -0.031238
dtype: float64

In [27]:
dFC_weights.loc[~np.isclose(dFC_weights, 0)].sort_values(ascending=False, key=np.abs)

rsfmri_c_ngd_smh_ngd_ad       -0.473566
rsfmri_c_ngd_smm_ngd_smh      -0.441655
rsfmri_c_ngd_rspltp_ngd_cgc   -0.407215
rsfmri_c_ngd_ca_ngd_ad         0.322567
rsfmri_c_ngd_fo_ngd_fo         0.263529
rsfmri_c_ngd_smm_ngd_ca        0.239713
rsfmri_c_ngd_fo_ngd_dla        0.184051
rsfmri_c_ngd_sa_ngd_rspltp    -0.170718
rsfmri_c_ngd_smh_ngd_smh      -0.170622
rsfmri_c_ngd_ad_ngd_ad        -0.168985
rsfmri_c_ngd_smm_ngd_ad       -0.137745
rsfmri_c_ngd_smh_ngd_dla       0.137559
rsfmri_c_ngd_vs_ngd_smh        0.097477
rsfmri_c_ngd_sa_ngd_fo        -0.080852
rsfmri_c_ngd_smh_ngd_rspltp   -0.048507
rsfmri_c_ngd_vs_ngd_smm        0.045063
rsfmri_c_ngd_rspltp_ngd_ca    -0.044360
rsfmri_c_ngd_vta_ngd_vta       0.038087
rsfmri_c_ngd_smh_ngd_ca        0.033311
rsfmri_c_ngd_vta_ngd_cgc       0.016867
rsfmri_c_ngd_sa_ngd_ad         0.008383
rsfmri_c_ngd_vta_ngd_ca        0.001871
dtype: float64